# Linear classification


The classes are not linearly separable for the second classifier. The month of July has rain in Sydney but not Paris. The month of December has sun in Sydney but not Paris. A simple perceptron cannot handle the case of a XOR function. 

For the first classifier, we know that if we look at Gothenburg there will always be rain, so the classifier only need to figure out which month we're in to decided the weather for Paris. The inputs matters for the classifiers. 

In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline

X1 = [{'city':'Gothenburg', 'month':'July'},
      {'city':'Gothenburg', 'month':'December'},
      {'city':'Paris', 'month':'July'},
      {'city':'Paris', 'month':'December'}]
Y1 = ['rain', 'rain', 'sun', 'rain']

X2 = [{'city':'Sydney', 'month':'July'},
      {'city':'Sydney', 'month':'December'},
      {'city':'Paris', 'month':'July'},
      {'city':'Paris', 'month':'December'}]
Y2 = ['rain', 'sun', 'sun', 'rain']

classifier1 = make_pipeline(DictVectorizer(), Perceptron(max_iter=10))
classifier1.fit(X1, Y1)
guesses1 = classifier1.predict(X1)
print(accuracy_score(Y1, guesses1))

classifier2 = make_pipeline(DictVectorizer(), Perceptron(max_iter=10))
#classifier2 = make_pipeline(DictVectorizer(), LinearSVC())
classifier2.fit(X2, Y2)
guesses2 = classifier2.predict(X2)
print(accuracy_score(Y2, guesses2))


In [ ]:
import numpy as np
from sklearn.base import BaseEstimator

class LinearClassifier(BaseEstimator):
    """
    General class for binary linear classifiers. Implements the predict
    function, which is the same for all binary linear classifiers. There are
    also two utility functions.
    """

    def decision_function(self, X):
        """
        Computes the decision function for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """
        return X.dot(self.w)

    def predict(self, X):
        """
        Predicts the outputs for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """

        # First compute the output scores
        scores = self.decision_function(X)

        # Select the positive or negative class label, depending on whether
        # the score was positive or negative.
        out = np.select([scores >= 0.0, scores < 0.0],
                        [self.positive_class,
                         self.negative_class])
        return out

    def find_classes(self, Y):
        """
        Finds the set of output classes in the output part Y of the training set.
        If there are exactly two classes, one of them is associated to positive
        classifier scores, the other one to negative scores. If the number of
        classes is not 2, an error is raised.
        """
        classes = sorted(set(Y))
        if len(classes) != 2:
            raise Exception("this does not seem to be a 2-class problem")
        self.positive_class = classes[1]
        self.negative_class = classes[0]

    def encode_outputs(self, Y):
        """
        A helper function that converts all outputs to +1 or -1.
        """
        return np.array([1 if y == self.positive_class else -1 for y in Y])

In [ ]:
class Perceptron(LinearClassifier):
    """
    A straightforward implementation of the perceptron learning algorithm.
    """

    def __init__(self, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)

        # Perceptron algorithm:
        for i in range(self.n_iter):
            for x, y in zip(X, Ye):

                # Compute the output score for this instance.
                score = x.dot(self.w)

                # If there was an error, update the weights.
                if y*score <= 0:
                    self.w += y*x

In [ ]:
##### The following part is for the optional task.

### Sparse and dense vectors don't collaborate very well in NumPy/SciPy.
### Here are two utility functions that help us carry out some vector
### operations that we'll need.

def add_sparse_to_dense(x, w, factor):
    """
    Adds a sparse vector x, scaled by some factor, to a dense vector.
    This can be seen as the equivalent of w += factor * x when x is a dense
    vector.
    """
    w[x.indices] += factor * x.data

def sparse_dense_dot(x, w):
    """
    Computes the dot product between a sparse vector x and a dense vector w.
    """
    return np.dot(w[x.indices], x.data)

Pegasos SVC implementation

In [ ]:
class Pegasos_SVC(LinearClassifier):
    """
    Implementation of Pegasos SVC.
    """

    def __init__(self, regularization=0, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.regularization = regularization
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)

        # Initialize counter
        t = 0

        # Pegasos algo:
        for i in range(self.n_iter):
            # Initialize the sum of loss values
            loss_sum = 0
            for x, y in zip(X, Ye):
                t += 1
                eta = 1/(self.regularization*t)

                # Compute the output score for this instance.
                score = x.dot(self.w)

                # If y*score is less than 1, update weight vector with gradient of the hinge loss
                if y*score < 1:
                    self.w = (1-eta*self.regularization)*self.w+eta*y*x
                    # Update the sum of loss values
                    loss_sum += 1-y*score
                # Else, update weight vector with regularization
                else:
                    self.w *= (1-eta*self.regularization)

            # Print current epoch and value of the objective function
            print(f"Epoch {i+1}, Value of obj.function: {(loss_sum/len(X))+self.regularization}")
        print("-----------")

Logistic regression implementation

In [31]:
class LR(LinearClassifier):
    """
    Implementation of Logistic regression.
    """

    def __init__(self, regularization=0, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.regularization = regularization
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)

        # Initialize counter
        t = 0

        # LR algo algorithm:
        for i in range(self.n_iter):
            # Initialize the sum of loss values
            loss_sum = 0
            for x, y in zip(X, Ye):
                t += 1
                eta = 1/(self.regularization*t)

                # Compute score for this instance
                score = x.dot(self.w)

                # Gradient
                gradient = 1/(1+np.exp(y*score))
                # Update the weight vector
                self.w = (1-eta*self.regularization)*self.w+eta*y*gradient*x

                # Update the sum of loss values
                loss_sum += -np.log(1-gradient)
        
            # Print current epoch and value of the objective function
            print(f"Epoch {i+1}, Value of obj.function: {(loss_sum/len(X))+self.regularization}")
        print("-----------")

### Bonus task 1

a) BLAS implementation

In [ ]:
from scipy.linalg import blas
class blas_Pegasos_SVC(LinearClassifier):
    """
    Implementation of Pegasos SVC with BLAS functions.
    """

    def __init__(self, regularization=0, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.regularization = regularization
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)

        # Initialize counter
        t = 0

        # Pegasos algo:
        for i in range(self.n_iter):
            # Initialize the sum of loss values
            loss_sum = 0
            for x, y in zip(X, Ye):
                t += 1
                eta = 1/(self.regularization*t)

                # Compute the output score for this instance.
                score = blas.ddot(x, self.w)

                # If y*score is less than 1, update weight vector with gradient of the hinge loss
                if y*score < 1:
                    # Scale weight vector
                    y_a = blas.dscal((1-eta*self.regularization), self.w)
                    # Update weight vector
                    self.w = blas.daxpy(x, y_a, a = (eta*y))
                    # Update the sum of loss values
                    loss_sum += 1-y*score
                # Else, update weight vector without gradient
                else:
                    self.w = blas.dscal((1-eta*self.regularization), self.w)

            # Print current epoch and value of the objective function
            print(f"Epoch {i+1}, Value of obj.function: {(loss_sum/len(X))+self.regularization}")
        print("-----------")

b & c) Sparse vectors implementation with rescaling factor

In [ ]:
class sparse_Pegasos_SVC(LinearClassifier):
    """
    Implementation of Pegasos SVC using sparse vectors.
    """

    def __init__(self, regularization=0, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.regularization = regularization
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the Pegasos SVC learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # Initialize the weight vector to all zeros.
        self.w = np.zeros(X.shape[1])

        # Iteration through sparse matrices can be a bit slow, so we first
        # prepare this list to speed up iteration.
        XY = list(zip(X, Ye))
        print(self.w.shape)
        # Initialize counter
        t = 0
        # Pegasos algo:
        for i in range(self.n_iter):
            # Initialize the sum of loss values
            loss_sum = 0

            for x, y in XY:
                t += 1
                # Learning rate
                eta = 1/(self.regularization*t)
                # Compute the output score for this instance.
                score = sparse_dense_dot(x, self.w)

                # If y*score is less than 1, update weight vector with gradient of the hinge loss
                if y*score < 1:
                    self.w *= (1-eta*self.regularization)
                    add_sparse_to_dense(x, self.w, (eta*y))
                    # Update the sum of loss values
                    loss_sum += 1-y*score
                # Else, update weight vector without gradient
                else:
                    self.w *= (1-eta*self.regularization)

            # Print current epoch and value of the objective function
            print(f"Epoch {i+1}, Value of obj.function: {(loss_sum/X.shape[0])+self.regularization}")
        print("-----------")

In [ ]:
class sparse_Pegasos_SVC_scaling(LinearClassifier):
    """
    Implementation of Pegasos SVC using sparse veectors and a rescaling operation.
    """

    def __init__(self, regularization=0, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.regularization = regularization
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the Pegasos SVC learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # Initialize the weight vector to all zeros.
        self.w = np.zeros(X.shape[1])

        # Iteration through sparse matrices can be a bit slow, so we first
        # prepare this list to speed up iteration.
        XY = list(zip(X, Ye))

        # Initialize counter
        t = 0
        # Pegasos algo:
        for i in range(self.n_iter):
            # Initialize the sum of loss values
            loss_sum = 0
            # Initialize scaling factor
            a = 1

            for x, y in XY:
                t += 1
                # Learning rate
                eta = 1/(self.regularization*t)
                # Compute the output score for this instance.
                score = sparse_dense_dot(x, self.w)*a

                # If y*score is less than 1, update weight vector with gradient of the hinge loss
                if y*score < 1:
                    a *= (1-eta*self.regularization)
                    # Check for ZeroDivisionError
                    if a != 0:
                        add_sparse_to_dense(x, self.w, ((eta*y)/a))
                    else:
                        # If a was 0, set it to some small default value
                        add_sparse_to_dense(x, self.w, eta*y/0.00001)
                    # Update the sum of loss values
                    loss_sum += 1-y*score
                # Else, update weight vector without gradient
                else:
                    a *= (1-eta*self.regularization)
            
            # Postponed scaling operation   
            self.w *= a

            # Print current epoch and value of the objective function
            print(f"Epoch {i+1}, Value of obj.function: {(loss_sum/X.shape[0])+self.regularization}")
        print("-----------")

Doc classification 

In [ ]:
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
# This function reads the corpus, returns a list of documents, and a list
# of their corresponding polarity labels. 
def read_data(corpus_file):
    X = []
    Y = []
    with open(corpus_file, encoding='utf-8') as f:
        for line in f:
            _, y, _, x = line.split(maxsplit=3)
            X.append(x.strip())
            Y.append(y)
    return X, Y

In [35]:
# Read all the documents.
X, Y = read_data('data/all_sentiment_shuffled.txt')

# Split into training and test parts.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                random_state=0)

# Set up the preprocessing steps and the classifier.
pipeline = make_pipeline(
    TfidfVectorizer(ngram_range=(1,1)),
    SelectKBest(k=1000),
    Normalizer(),

    # NB that this is our Perceptron, not sklearn.linear_model.Perceptron
    #Perceptron()
    
    Pegasos_SVC(regularization=1/len(Ytrain), n_iter=20)
    #LR(regularization=(1/len(Ytrain)), n_iter=20)

    # Bonus
    #blas_Pegasos_SVC(regularization=1/len(Ytrain), n_iter=50)
    #sparse_Pegasos_SVC(regularization=1/len(Ytrain), n_iter=20)
    #sparse_Pegasos_SVC_scaling(regularization=1/len(Ytrain), n_iter=20)
)

# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))

# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Epoch 1, Value of obj.function: 1.1909944377665065
Epoch 2, Value of obj.function: 0.37185947122254825
Epoch 3, Value of obj.function: 0.34274596029431936
Epoch 4, Value of obj.function: 0.32858046666425966
Epoch 5, Value of obj.function: 0.32156240203756964
Epoch 6, Value of obj.function: 0.3157702532831923
Epoch 7, Value of obj.function: 0.3123038380929444
Epoch 8, Value of obj.function: 0.3093708626908032
Epoch 9, Value of obj.function: 0.30725373333243494
Epoch 10, Value of obj.function: 0.3051811171838598
Epoch 11, Value of obj.function: 0.3038702235797561
Epoch 12, Value of obj.function: 0.30255867351836885
Epoch 13, Value of obj.function: 0.3016966598536792
Epoch 14, Value of obj.function: 0.30051354789804713
Epoch 15, Value of obj.function: 0.2998919253892097
Epoch 16, Value of obj.function: 0.2990332323534156
Epoch 17, Value of obj.function: 0.29872793812690696
Epoch 18, Value of obj.function: 0.2980287376510177
Epoch 19, Value of obj.function: 0.29764080275412547
Epoch 20, Va

### Pegasos SVC and Logistic regression comparison

Both implementations were run with the "default" settings, SelectKBest(1000), ngram_range=(1,1), lambda=1/n and we iterated 20 times through the training set during training.

Pegasos SVC 0.8317 accuracy (20 iterations, 3.95s)
LR 0.8326 accuracy (20 iterations, 7.87s)

## Time comparisons

### BLAS implementation for SVC vs regular:

We ran the regular Pegasos implementation for 50 iteration and received an accuracy of approximately 83%. The training time was a total of 8.21 seconds. With the BLAS implementation we received exactly the same accuracy but the training time had decreased to 5.62 seconds. 

### Sparse vectors vs regular:

For this we first removed the SelectKBest step from the pipeline and changed the ngram_range parameter of the TfidfVectorizer to (1,2). This significantly increases the number of features, from 1000 features to 480514 features. However, we did not manage to run the initial implementation of the Pegasos SVC with this many features. Therefore we changed the ngram_range parameter back to (1,1), 

With the sparse implementation the training time for 20 iterations was 47.07s and the accuracy was 0.8695.

### Sparse vectors + speeding up the scaling operation:

When we implemented the rescaling operation the training time for 20 iterations decreased significantly to 10.71s and the accuracy stayed around the same.